In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
from matplotlib import pyplot as plt
import time

# Using mediapipe holistic, full body pose estimation

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,  mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_custom_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,  mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(255, 255, 204), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(225, 225, 225), thickness=1, circle_radius=1))
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(204, 153, 255), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0, 128, 0), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))
    

# Get landmarks keypoints from frames

In [6]:
def get_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
  
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3) 

    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, lh, rh])

# Load data

In [7]:
DATA_PATH = os.path.join('points_Data')

actions = np.array(['hello', 'thank you', 'i am', 'w', 'a', 'L'])
#['##', hello', 'thank you', 'i love you', 'i am', 'w', 'a', 'i', 'l']
no_sequences = 50

sequence_length = 30

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
actions

array(['hello', 'thank you', 'i am', 'w', 'a', 'L'], dtype='<U9')

In [11]:
sequences, labels = [], []

for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in  range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)

In [14]:
len(sequences)

300

# Tensor Flow Model

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model

In [16]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [17]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [19]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
9/9 [==============================] - 4s 70ms/step - loss: 1.9976 - categorical_accuracy: 0.1614
Epoch 2/2000
9/9 [==============================] - 1s 71ms/step - loss: 1.8348 - categorical_accuracy: 0.2421
Epoch 3/2000
9/9 [==============================] - 1s 71ms/step - loss: 1.6882 - categorical_accuracy: 0.2842
Epoch 4/2000
9/9 [==============================] - 1s 72ms/step - loss: 1.2186 - categorical_accuracy: 0.4140
Epoch 5/2000
9/9 [==============================] - 1s 71ms/step - loss: 1.1554 - categorical_accuracy: 0.4456
Epoch 6/2000
9/9 [==============================] - 1s 74ms/step - loss: 1.1293 - categorical_accuracy: 0.4140
Epoch 7/2000
9/9 [==============================] - 1s 72ms/step - loss: 0.9307 - categorical_accuracy: 0.5404
Epoch 8/2000
9/9 [==============================] - 1s 75ms/step - loss: 0.9344 - categorical_accuracy: 0.5263
Epoch 9/2000
9/9 [==============================] - 1s 74ms/step - loss: 0.7901 - categorical_accuracy: 0.6175
E

9/9 [==============================] - 1s 85ms/step - loss: 0.7560 - categorical_accuracy: 0.7158
Epoch 75/2000
9/9 [==============================] - 1s 85ms/step - loss: 0.9112 - categorical_accuracy: 0.5053
Epoch 76/2000
9/9 [==============================] - 1s 83ms/step - loss: 0.6909 - categorical_accuracy: 0.6491
Epoch 77/2000
9/9 [==============================] - 1s 85ms/step - loss: 3.4502 - categorical_accuracy: 0.4175
Epoch 78/2000
9/9 [==============================] - 1s 85ms/step - loss: 2.0255 - categorical_accuracy: 0.1895
Epoch 79/2000
9/9 [==============================] - 1s 86ms/step - loss: 1.3748 - categorical_accuracy: 0.3158
Epoch 80/2000
9/9 [==============================] - 1s 88ms/step - loss: 1.3769 - categorical_accuracy: 0.3368
Epoch 81/2000
9/9 [==============================] - 1s 89ms/step - loss: 1.4109 - categorical_accuracy: 0.2842
Epoch 82/2000
9/9 [==============================] - 1s 85ms/step - loss: 1.2379 - categorical_accuracy: 0.3158
Epoch 

9/9 [==============================] - 1s 89ms/step - loss: 0.1495 - categorical_accuracy: 0.9684
Epoch 147/2000
9/9 [==============================] - 1s 86ms/step - loss: 0.1271 - categorical_accuracy: 0.9684
Epoch 148/2000
9/9 [==============================] - 1s 87ms/step - loss: 0.1075 - categorical_accuracy: 0.9684
Epoch 149/2000
9/9 [==============================] - 1s 87ms/step - loss: 0.0966 - categorical_accuracy: 0.9684
Epoch 150/2000
9/9 [==============================] - 1s 88ms/step - loss: 0.4300 - categorical_accuracy: 0.8456
Epoch 151/2000
9/9 [==============================] - 1s 87ms/step - loss: 0.6253 - categorical_accuracy: 0.7579
Epoch 152/2000
9/9 [==============================] - 1s 86ms/step - loss: 0.4238 - categorical_accuracy: 0.7825
Epoch 153/2000
9/9 [==============================] - 1s 87ms/step - loss: 0.2440 - categorical_accuracy: 0.9088
Epoch 154/2000
9/9 [==============================] - 1s 87ms/step - loss: 0.2492 - categorical_accuracy: 0.863

9/9 [==============================] - 1s 92ms/step - loss: 0.4289 - categorical_accuracy: 0.8632
Epoch 219/2000
9/9 [==============================] - 1s 94ms/step - loss: 1.2553 - categorical_accuracy: 0.6386
Epoch 220/2000
9/9 [==============================] - 1s 93ms/step - loss: 1.6422 - categorical_accuracy: 0.4281
Epoch 221/2000
9/9 [==============================] - 1s 92ms/step - loss: 1.1354 - categorical_accuracy: 0.4737
Epoch 222/2000
9/9 [==============================] - 1s 90ms/step - loss: 0.9366 - categorical_accuracy: 0.5649
Epoch 223/2000
9/9 [==============================] - 1s 93ms/step - loss: 0.9602 - categorical_accuracy: 0.5263
Epoch 224/2000
9/9 [==============================] - 1s 91ms/step - loss: 0.8461 - categorical_accuracy: 0.5895
Epoch 225/2000
9/9 [==============================] - 1s 92ms/step - loss: 0.6951 - categorical_accuracy: 0.6596
Epoch 226/2000
9/9 [==============================] - 1s 92ms/step - loss: 0.6994 - categorical_accuracy: 0.691

9/9 [==============================] - 1s 104ms/step - loss: 0.1473 - categorical_accuracy: 0.9509
Epoch 291/2000
9/9 [==============================] - 1s 105ms/step - loss: 0.1681 - categorical_accuracy: 0.9404
Epoch 292/2000
9/9 [==============================] - 1s 103ms/step - loss: 0.1291 - categorical_accuracy: 0.9579
Epoch 293/2000
9/9 [==============================] - 1s 104ms/step - loss: 0.1419 - categorical_accuracy: 0.9474
Epoch 294/2000
9/9 [==============================] - 1s 105ms/step - loss: 0.1643 - categorical_accuracy: 0.9333
Epoch 295/2000
9/9 [==============================] - 1s 105ms/step - loss: 0.1209 - categorical_accuracy: 0.9439
Epoch 296/2000
9/9 [==============================] - 1s 107ms/step - loss: 0.1152 - categorical_accuracy: 0.9579
Epoch 297/2000
9/9 [==============================] - 1s 109ms/step - loss: 0.0929 - categorical_accuracy: 0.9614
Epoch 298/2000
9/9 [==============================] - 1s 110ms/step - loss: 0.0560 - categorical_accura

...

Epoch 1699/2000
9/9 [==============================] - 1s 143ms/step - loss: 6.1133e-05 - categorical_accuracy: 1.0000
Epoch 1700/2000
9/9 [==============================] - 1s 143ms/step - loss: 6.0316e-05 - categorical_accuracy: 1.0000
Epoch 1701/2000
9/9 [==============================] - 1s 142ms/step - loss: 5.9796e-05 - categorical_accuracy: 1.0000
Epoch 1702/2000
9/9 [==============================] - 1s 142ms/step - loss: 5.8502e-05 - categorical_accuracy: 1.0000
Epoch 1703/2000
9/9 [==============================] - 1s 139ms/step - loss: 5.7999e-05 - categorical_accuracy: 1.0000
Epoch 1704/2000
9/9 [==============================] - 1s 141ms/step - loss: 5.7024e-05 - categorical_accuracy: 1.0000
Epoch 1705/2000
9/9 [==============================] - 1s 139ms/step - loss: 5.6160e-05 - categorical_accuracy: 1.0000
Epoch 1706/2000
9/9 [==============================] - 1s 140ms/step - loss: 5.5551e-05 - categorical_accuracy: 1.0000
Epoch 1707/2000
9/9 [===========================

9/9 [==============================] - 1s 140ms/step - loss: 2.8719e-05 - categorical_accuracy: 1.0000
Epoch 1768/2000
9/9 [==============================] - 1s 140ms/step - loss: 2.8409e-05 - categorical_accuracy: 1.0000
Epoch 1769/2000
9/9 [==============================] - 1s 141ms/step - loss: 2.8139e-05 - categorical_accuracy: 1.0000
Epoch 1770/2000
9/9 [==============================] - 1s 140ms/step - loss: 2.7930e-05 - categorical_accuracy: 1.0000
Epoch 1771/2000
9/9 [==============================] - 1s 140ms/step - loss: 2.7778e-05 - categorical_accuracy: 1.0000
Epoch 1772/2000
9/9 [==============================] - 1s 140ms/step - loss: 2.7339e-05 - categorical_accuracy: 1.0000
Epoch 1773/2000
9/9 [==============================] - 1s 141ms/step - loss: 2.7232e-05 - categorical_accuracy: 1.0000
Epoch 1774/2000
9/9 [==============================] - 1s 139ms/step - loss: 2.6970e-05 - categorical_accuracy: 1.0000
Epoch 1775/2000
9/9 [==============================] - 1s 138ms/

Epoch 1836/2000
9/9 [==============================] - 1s 150ms/step - loss: 1.5186e-05 - categorical_accuracy: 1.0000
Epoch 1837/2000
9/9 [==============================] - 1s 151ms/step - loss: 1.5059e-05 - categorical_accuracy: 1.0000
Epoch 1838/2000
9/9 [==============================] - 1s 154ms/step - loss: 1.4920e-05 - categorical_accuracy: 1.0000
Epoch 1839/2000
9/9 [==============================] - 1s 147ms/step - loss: 1.4809e-05 - categorical_accuracy: 1.0000
Epoch 1840/2000
9/9 [==============================] - 1s 144ms/step - loss: 1.4699e-05 - categorical_accuracy: 1.0000
Epoch 1841/2000
9/9 [==============================] - 1s 143ms/step - loss: 1.4541e-05 - categorical_accuracy: 1.0000
Epoch 1842/2000
9/9 [==============================] - 1s 143ms/step - loss: 1.4431e-05 - categorical_accuracy: 1.0000
Epoch 1843/2000
9/9 [==============================] - 1s 144ms/step - loss: 1.4320e-05 - categorical_accuracy: 1.0000
Epoch 1844/2000
9/9 [===========================

9/9 [==============================] - 1s 149ms/step - loss: 9.0872e-06 - categorical_accuracy: 1.0000
Epoch 1905/2000
9/9 [==============================] - 1s 148ms/step - loss: 9.0173e-06 - categorical_accuracy: 1.0000
Epoch 1906/2000
9/9 [==============================] - 1s 146ms/step - loss: 8.9563e-06 - categorical_accuracy: 1.0000
Epoch 1907/2000
9/9 [==============================] - 1s 146ms/step - loss: 8.8902e-06 - categorical_accuracy: 1.0000
Epoch 1908/2000
9/9 [==============================] - 1s 147ms/step - loss: 8.8467e-06 - categorical_accuracy: 1.0000
Epoch 1909/2000
9/9 [==============================] - 1s 148ms/step - loss: 8.7698e-06 - categorical_accuracy: 1.0000
Epoch 1910/2000
9/9 [==============================] - 1s 148ms/step - loss: 8.7104e-06 - categorical_accuracy: 1.0000
Epoch 1911/2000
9/9 [==============================] - 1s 148ms/step - loss: 8.6372e-06 - categorical_accuracy: 1.0000
Epoch 1912/2000
9/9 [==============================] - 1s 148ms/

Epoch 1973/2000
9/9 [==============================] - 1s 146ms/step - loss: 5.7379e-06 - categorical_accuracy: 1.0000
Epoch 1974/2000
9/9 [==============================] - 1s 148ms/step - loss: 5.7233e-06 - categorical_accuracy: 1.0000
Epoch 1975/2000
9/9 [==============================] - 1s 147ms/step - loss: 5.6472e-06 - categorical_accuracy: 1.0000
Epoch 1976/2000
9/9 [==============================] - 1s 146ms/step - loss: 5.6016e-06 - categorical_accuracy: 1.0000
Epoch 1977/2000
9/9 [==============================] - 1s 146ms/step - loss: 5.5807e-06 - categorical_accuracy: 1.0000
Epoch 1978/2000
9/9 [==============================] - 1s 145ms/step - loss: 5.5531e-06 - categorical_accuracy: 1.0000
Epoch 1979/2000
9/9 [==============================] - 1s 146ms/step - loss: 5.5104e-06 - categorical_accuracy: 1.0000
Epoch 1980/2000
9/9 [==============================] - 1s 145ms/step - loss: 5.4690e-06 - categorical_accuracy: 1.0000
Epoch 1981/2000
9/9 [===========================

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 596774 (2.28 MB)
Trainable params: 596774 

In [52]:
res = model.predict(X_train)
len(res)

9/9 [==============================] - 0s 30ms/step


285

In [39]:
actions[np.argmax(res[0])]

'i am'

In [40]:
actions[np.argmax(y_test[0])]

'i am'

In [24]:
model.save('sign.keras')

In [41]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [42]:
yhat = model.predict(X_train)

9/9 [==============================] - 0s 23ms/step


In [43]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [44]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[238,   0],
        [  0,  47]],

       [[239,   0],
        [  0,  46]],

       [[236,   0],
        [  0,  49]],

       [[238,   0],
        [  0,  47]],

       [[238,   0],
        [  0,  47]],

       [[236,   0],
        [  0,  49]]], dtype=int64)

In [45]:
accuracy_score(ytrue, yhat)

1.0

In [57]:
colors = [(245,117,16), (117,245,16), (16,117,245),(16,117,22),(21,23,245),(16,22,33)]
#, (16,117,245),(21,23,245),(16,22,33),(16,117,22),(16,0,245),(23,117,245)
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [58]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

TypeError: only size-1 arrays can be converted to Python scalars

<Figure size 1800x1800 with 0 Axes>

# Test the model

In [33]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from tensorflow.keras.models import load_model
model=load_model('sign.keras')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 596774 (2.28 MB)
Trainable params: 596774 

In [65]:
sequence = []
sentence = []
threshold = 0.8


cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    
    prev_keypoints = None  # Store the previous keypoints
    
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        image, results = mediapipe_detection(frame, holistic)
         
        draw_custom_landmarks(image, results)
        
        # Predict only when there's a significant change in keypoints
        keypoints = get_keypoints(results)
        
        if prev_keypoints is not None:
            # Calculate the Euclidean distance between the current and previous keypoints
            euclidean_distance = np.linalg.norm(np.array(prev_keypoints) - np.array(keypoints))
            #print(euclidean_distance)
            # You can set a threshold for the distance below which you don't make predictions
            if euclidean_distance > 0.2:
                sequence.append(keypoints)
                sequence = sequence[-30:]
        
        prev_keypoints = keypoints
        
        
        #predict
        
#         keypoints = get_keypoints(results)
#         sequence.append(keypoints)
#         sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
        if res.any():
            if res[np.argmax(res)].any() > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                        
                else:
                    sentence.append(actions[np.argmax(res)])
                  

    
        if len(sentence) > 5:
            sentence = sentence[-5:]
        # Viz probabilities
        
        image = prob_viz(res, actions, image, colors)
            
        
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        cv2.imshow("Camera", image)
        
        if cv2.waitKey(10) & 0XFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()
    

In [60]:
cap.release()
cv2.destroyAllWindows()